In [4]:
import numpy as np
import pandas as pd;
import math as mt
import sys
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder;
from sklearn.decomposition import PCA;
import matplotlib.pyplot as plt;

In [17]:
class XMeans:
    def loglikelihood(self, r, rn, var, m, k):
        l1 = - rn / 2.0 * mt.log(2 * mt.pi)
        l2 = - rn * m / 2.0 * mt.log(var)
        l3 = - (rn - k) / 2.0
        l4 = rn * mt.log(rn)
        l5 = - rn * mt.log(r)

        return l1 + l2 + l3 + l4 + l5

    def __init__(self, X, kmax = 20):
        self.X = X
        self.num = np.size(self.X, axis=0)
        self.dim = np.size(X, axis=1)
        self.KMax = kmax

    def fit(self):
        k = 1
        X = self.X
        M = self.dim
        num = self.num

        while(1):
            ok = k

            #Improve Params
            kmeans = KMeans(n_clusters=k).fit(X)
            labels = kmeans.labels_
            m = kmeans.cluster_centers_

            #Improve Structure
            #Calculate BIC
            p = M + 1

            obic = np.zeros(k)

            for i in range(k):
                rn = np.size(np.where(labels == i))
                var = np.sum((X[labels == i] - m[i])**2)/float(rn - 1)
                obic[i] = self.loglikelihood(rn, rn, var, M, 1) - p/2.0*mt.log(rn)

            #Split each cluster into two subclusters and calculate BIC of each splitted cluster
            sk = 2 #The number of subclusters
            nbic = np.zeros(k)
            addk = 0

            for i in range(k):
                ci = X[labels == i]
                r = np.size(np.where(labels == i))

                kmeans = KMeans(n_clusters=sk).fit(ci)
                ci_labels = kmeans.labels_
                sm = kmeans.cluster_centers_

                for l in range(sk):
                    rn = np.size(np.where(ci_labels == l))
                    var = np.sum((ci[ci_labels == l] - sm[l])**2)/float(rn - sk)
                    nbic[i] += self.loglikelihood(r, rn, var, M, sk)

                p = sk * (M + 1)
                nbic[i] -= p/2.0*mt.log(r)

                if obic[i] < nbic[i]:
                    addk += 1

            k += addk

            # print(obic)
            # print(nbic)

            if ok == k or k >= self.KMax:
                break


        #Calculate labels and centroids
        kmeans = KMeans(n_clusters=k).fit(X)
        self.labels = kmeans.labels_
        self.k = k
        self.m = kmeans.cluster_centers_

In [18]:
%config IPCompleter.greedy=True

In [19]:
df = pd.read_csv('~/Master Thesis/Data/iris.csv')

In [20]:
enc = LabelEncoder();
enc.fit(df['species']);
df['species'] = enc.transform(df['species'])

In [21]:
pca = PCA(n_components=3);
pca.fit(df)
df_2d=pca.transform(df)
print(pca.explained_variance_ratio_)

[0.92264045 0.04810411 0.01829962]


In [26]:
sample=np.array(df_2d)
sample

array([[-2.86550669e+00,  3.02780442e-01,  4.84897211e-02],
       [-2.89321339e+00, -1.76987146e-01, -1.68798182e-01],
       [-3.05574910e+00, -1.68006289e-01,  5.65424149e-02],
       [-2.92124902e+00, -3.25027211e-01,  9.80010810e-03],
       [-2.90675140e+00,  3.02332724e-01,  1.53720302e-01],
       [-2.48899988e+00,  7.39727187e-01,  2.00679240e-01],
       [-2.99106535e+00, -1.16739834e-01,  2.50385821e-01],
       [-2.81105291e+00,  1.55635814e-01,  1.66268489e-02],
       [-3.05248032e+00, -5.89687583e-01, -9.28733240e-03],
       [-2.85470148e+00, -1.11633294e-01, -1.64955858e-01],
       [-2.70085679e+00,  6.36298840e-01,  2.34535897e-02],
       [-2.79784383e+00,  8.04346801e-03,  8.99945580e-02],
       [-2.96034027e+00, -2.39025333e-01, -1.67059861e-01],
       [-3.36757216e+00, -5.53686579e-01,  9.81963025e-02],
       [-2.83079955e+00,  1.14659075e+00,  7.49186344e-02],
       [-2.58796715e+00,  1.30810539e+00,  4.03602440e-01],
       [-2.80918503e+00,  7.79232365e-01

In [30]:
#Generate data
X, TrueLabels = datasets.make_blobs(n_samples=10,n_features=3)
X

array([[-5.20907174, -8.80785691,  4.60301755],
       [ 7.42772106, -8.55930187,  0.86706993],
       [ 9.88847375, -0.62532684,  4.60684856],
       [ 6.47421494, -9.0613259 , -0.38411683],
       [-5.28849797, -8.7521393 ,  6.19736211],
       [ 5.76266127, -9.55013447,  1.23015404],
       [-6.62319309, -8.92618457,  7.6025491 ],
       [11.13065214, -0.93531589,  4.68619149],
       [ 8.40610004, -0.8640962 ,  3.0424992 ],
       [11.76186429,  0.11608297,  3.87358253]])

In [31]:
x_means_instance = XMeans(X)
x_means_instance.fit()

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in double_scalars


ValueError: math domain error